# Sandbox

Herman Kamper, Stellenbosch University, 2019.

## Preliminaries

In [65]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from os import path
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tensorflow as tf

sys.path.append(path.join("..", "src"))
import tflego

from tflego import NP_DTYPE, TF_DTYPE, NP_ITYPE, TF_ITYPE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
import warnings
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
tf.logging.set_verbosity(tf.logging.ERROR)
if type(tf.contrib) != type(tf):
    tf.contrib._warning = None

## Conditioning

In [70]:
tf.reset_default_graph()

# Random seed
np.random.seed(1)
tf.set_random_seed(1)

# Parameters
n_data = 7
d_in = 5
n_languages = 3
n_classes = 2
test_data = np.asarray(np.random.randn(n_data, d_in), dtype=NP_DTYPE)
test_language = np.asarray([0, 0, 0, 1, 1, 2, 2], dtype=NP_ITYPE)  # want to split accordingly
test_class =    np.asarray([0, 1, 0, 1, 0, 0, 1], dtype=NP_ITYPE)  # output class

# Model
x = tf.placeholder(TF_DTYPE, [None, d_in])
language = tf.placeholder(TF_ITYPE, [None])
y = tf.placeholder(TF_ITYPE, [None])
ff = tflego.build_feedforward(
    x, [10, 9]
    )
split_networks = []
for i_lang in range(n_languages):
    with tf.variable_scope("split_{}".format(i_lang)):
        split_network = tflego.build_feedforward(
            ff, [6, n_classes]
            )
        if i_lang == 0:
            split_network *= 0
        elif i_lang == 1:
            split_network *= np.inf
        elif i_lang == 2:
            pass
    split_networks.append(split_network)
    
output = tf.where(tf.equal(language, 0), split_networks[0], 
    tf.where(tf.equal(language, 1), split_networks[1], split_networks[2])
    )

# output = tf.where(tf.equal(language, 0), (language + 1)*55, language*0)
# output = tf.where(tf.equal(language, 0), 55, 
#     tf.where(tf.equal(language, 1), 66,
#     tf.where(tf.equal(language, 2), 9, -1
#     )))


In [71]:
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    np_output = output.eval({x: test_data, language: test_language})

In [72]:
np_output

array([[-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [        inf,        -inf],
       [        inf,        -inf],
       [ 1.7773362 , -0.6447714 ],
       [ 1.8913155 , -0.20415437]], dtype=float32)